In [6]:
import pandas as pd

In [11]:
workforce_census_filepath = "../data/census/insee_work-census_2021/base-ic-activite-residents-2021.xlsx"
netincome_census_filepath = "../data/census/insee_net-income_2021/BASE_TD_FILO_IRIS_2021_DEC.xlsx"
disposableincome_census_filepath = "../data/census/insee_disposable-income_2021/BASE_TD_FILO_IRIS_2021_DISP.xlsx"

workforce_census = pd.read_excel(workforce_census_filepath, sheet_name='IRIS')
netincome_census = pd.read_excel(netincome_census_filepath, sheet_name='IRIS')
disposableincome_census = pd.read_excel(disposableincome_census_filepath, sheet_name='IRIS')

In [93]:
wf_df = workforce_census.copy()
ni_df = netincome_census.copy()
di_df = disposableincome_census.copy()

# Start testing

In [94]:
# compute worforce census rates as % of population
rate_definitions = {
    "unemployment_rate": ("P21_CHOM1564", "P21_ACT1564"),
    "employed_rate": ("P21_ACTOCC1564", "P21_ACT1564"),
    "studying_rate": ("P21_ETUD1564", "P21_ACT1564"),
    "retired_rate": ("P21_RETR1564", "P21_POP1564"),
    "work_local": ("P21_ACTOCC15P_ILT1", "P21_ACTOCC1564"),
    "work_diffcommune_samearr": ("P21_ACTOCC15P_ILT2P", "P21_ACTOCC1564"),
    "work_diffcommune_samedept": ("P21_ACTOCC15P_ILT2", "P21_ACTOCC1564"),
    "work_diffcommune_diffdept_sameregion": ("P21_ACTOCC15P_ILT3", "P21_ACTOCC1564"),
    "work_diffregion": ("P21_ACTOCC15P_ILT4", "P21_ACTOCC1564"),
    "work_diffcountry_": ("P21_ACTOCC15P_ILT5", "P21_ACTOCC1564"),
    "commute_notransport": ("C21_ACTOCC15P_PAS", "P21_ACTOCC1564"),
    "commute_walk": ("C21_ACTOCC15P_MAR", "P21_ACTOCC1564"),
    "commute_bicycle": ("C21_ACTOCC15P_VELO", "P21_ACTOCC1564"),
    "commute_motorcycle": ("C21_ACTOCC15P_2ROUESMOT", "P21_ACTOCC1564"),
    "commute_vehicle": ("C21_ACTOCC15P_VOIT", "P21_ACTOCC1564"),
    "commute_publictransport": ("C21_ACTOCC15P_TCOM", "P21_ACTOCC1564"),
}

def add_rate_columns(df, rate_dict):
    for new_col, (num, denom) in rate_dict.items():
        df[new_col] = (df[num] / df[denom].replace(0, pd.NA)) * 100
    return df

# execute
wf_df = add_rate_columns(wf_df, rate_definitions)

In [95]:
# merge median net income into workforce census table
wf_df = wf_df.merge(ni_df[["IRIS", "DEC_MED21"]], on="IRIS", how="left")

# merge median disposable income into workforce census table
wf_df = wf_df.merge(di_df[["IRIS", "DISP_MED21"]], on="IRIS", how="left")

In [96]:
# rename columns
rename_map = {
    "P21_POP1564": "population",
    "DEC_MED21": "median_net_income",
    "DISP_MED21": "median_disposable_income",
    "IRIS": "INSEE_IRIS_ID"
}
wf_df = wf_df.rename(columns=rename_map)

In [97]:
# keep only columns of interest
rename_map.update({v[0]: k for k, v in rate_definitions.items()})
wf_df = wf_df[list(rename_map.values())]

In [98]:
# reorder dataframe columns
reorder_cols_list = ['INSEE_IRIS_ID', 'population', 'median_net_income',
 'median_disposable_income', 'unemployment_rate',
 'employed_rate',
 'studying_rate',
 'retired_rate',
 'work_local',
 'work_diffcommune_samearr',
 'work_diffcommune_samedept',
 'work_diffcommune_diffdept_sameregion',
 'work_diffregion',
 'work_diffcountry_',
 'commute_notransport',
 'commute_walk',
 'commute_bicycle',
 'commute_motorcycle',
 'commute_vehicle',
 'commute_publictransport']

wf_df = wf_df[reorder_cols_list]

In [99]:
# convert string values in income columns to NaNs
wf_df["median_net_income"] = pd.to_numeric(wf_df["median_net_income"], errors="coerce")
wf_df["median_disposable_income"] = pd.to_numeric(wf_df["median_disposable_income"], errors="coerce")
wf_df["population"] = wf_df["population"].round().astype("Int64")


In [100]:
wf_df

,INSEE_IRIS_ID,population,median_net_income,median_disposable_income,unemployment_rate,employed_rate,studying_rate,retired_rate,work_local,work_diffcommune_samearr,work_diffcommune_samedept,work_diffcommune_diffdept_sameregion,work_diffregion,work_diffcountry_,commute_notransport,commute_walk,commute_bicycle,commute_motorcycle,commute_vehicle,commute_publictransport
0,010010000,509,NaN,NaN,6.505535,93.494465,8.167972,7.744545,12.399781,88.830062,61.050549,24.547435,3.232078,0.0,4.984068,0.0,0.0,2.524759,102.973134,1.229844
1,010020000,163,NaN,NaN,1.649213,98.350787,14.145126,5.216707,17.785689,84.390547,73.09595,7.521109,3.773487,0.0,3.701542,0.0,0.0,0.0,80.666181,3.589818
2,010040101,1185,19330.0,20350.0,7.92546,92.07454,18.449277,3.222112,48.275906,53.328093,34.012401,17.540009,1.775682,0.0,7.723165,8.530443,2.696495,1.194611,76.568643,13.026165
3,010040102,2418,16830.0,18570.0,19.221104,80.778896,10.447223,4.038371,40.718848,60.514882,42.38568,17.019818,0.80922,0.300164,5.567385,9.340548,0.707352,1.962189,64.420833,16.779493
4,010040201,2649,19940.0,20700.0,14.209538,85.790462,9.83998,4.862404,43.293218,57.75102,41.386358,15.17145,1.193212,0.0,2.168272,12.58787,4.187983,0.862303,71.029556,14.784575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49275,974240101,1022,12030.0,14870.0,32.130744,67.869256,10.380073,5.032276,83.123301,18.012037,17.559771,0.0,0.452266,0.0,5.631322,29.533812,2.291124,0.0,65.094651,1.10866
49276,974240102,1081,10660.0,14230.0,36.044573,63.955427,9.113442,2.739557,83.756774,17.496079,17.496079,0.0,0.0,0.0,4.231377,15.917647,1.078749,0.0,65.981036,3.111633
49277,974240103,473,NaN,NaN,34.725478,65.274522,9.537947,2.463522,74.586819,26.378474,24.466923,0.0,1.911551,0.0,2.397027,11.830096,2.365857,0.0,84.100254,2.365857
49278,974240104,671,NaN,NaN,42.095045,57.904955,9.798959,3.131442,70.045529,32.042755,31.696986,0.0,0.34577,0.0,1.845775,14.152679,3.573399,0.0,62.099736,12.471218


In [101]:
# export to excel
wf_df.to_excel("../data/census/INSEE_census_2021.xlsx", index=False)

## Definitions
#### population
"P21_POP1564"

#### number active
"P21_ACT1564"

#### number unemployed
"P21_CHOM1564"

#### number employed
"C21_ACTOCC15P"

#### number studying
"P21_ETUD1564"

#### number retired
"P21_RETR1564"

#### work location
"P21_ACTOCC15P_ILT1" # number working in commune of residence
"P21_ACTOCC15P_ILT2P" # number working in a different commune
"P21_ACTOCC15P_ILT2" # number working in a different commune but in the same department as residence
"P21_ACTOCC15P_ILT3" # number working in a different commune & different department but in the same region as residence
"P21_ACTOCC15P_ILT4" # number working in a different region of France
"P21_ACTOCC15P_ILT5" # number working in a different country than France

#### commute method
"C21_ACTOCC15P_PAS" # number who do not use any transport for work
"C21_ACTOCC15P_MAR" # number commuting by walking
"C21_ACTOCC15P_VELO" # number commuting by bicycling
"C21_ACTOCC15P_2ROUESMOT" # number commuting by moped/scooter/motorcycle
"C21_ACTOCC15P_VOIT" # number commuting by car
"C21_ACTOCC15P_TCOM" # number commuting by public transport

#### income
"DEC_MED21" # median net income per capita
"DISP_MED21" # median disposable income per capita